In [3]:
import pygsheets
import pandas as pd
from tqdm import tqdm

In [4]:
client = pygsheets.authorize()

Please go to this URL and finish the authentication flow: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=798189642701-bad3c6vgo37u0au8bqegq1ei03fko9rj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=AhJugq96CaTYhQpCUC2y5bZQ6Ze4rd&prompt=consent&access_type=offline


In [5]:
workers_ids = ['W1', 'W2', 'W3', 'W4', 'W5', 'W6']
id_to_key_task1 = {
    'ind': {
        'W1': '14dAiIsBLxBUHzXgFDznMjSXCJmFY6qnIWOADqITWx9E',
        'W2': '1jHWRjwkahxpA5T_BWBRaMQ4q9kyg1l0gGz42pC6-dxI',
        'W3': '1C-Yyc17WFWScC5eu5WttQc8P8vCFuLvh8MqlBKiwF8c',
        'W4': '1J0xqeC05H1RVPSERi8fCXWUTGJ4iH3SI0w59BoFh_KU',
        'W5': '1T9jbfP1iapNLS94QZMiYw5K6LrZoP01Qf1jvTWpvdkU',
        'W6': '1G0chmXUmWC-lrZLg9ToQTGCdQmnbP0DA204bIKJn9yc',
    },
    'sun': {
        'W1': '1R-bT3RMu41cx-fnac3mNdb3OT_kb6vof1eMdCbU8_jM',
        'W2': '1ugtKZhO4jLtxW_yEwrrPXZtry2pLaP-6QgiIHKldl5E',
        'W3': '1OzP1XwzU3c-rNXyqxU3JKrrXHeXzSu3D6FIcfym4fHE',
        'W4': '1dNwW9dL4YPBEypUHQ6_-Dse8XL851W0WUiCBb28hRBQ',   
        'W5': '16wXGP08nESdLg4am4wfngeMNbK81IJAo3p0udg_iVlU',
        'W6': '1ZQEBOPGAlc6f2IUwavEKpPHY7dQxV4lN5iSQm1WHnX4',
    }
}
id_to_key_task2 = {
    'ind': {
        'W1': '1Wa33qUjeB0pq1QI87jUqHTXlQ88ADCxAFp77vFAhSVU',
        'W2': '1FbYeTu3ZK4vBLoPVpJqCFB8Rj6yzazYRpxRtmPXXf-k',
        'W3': '1UysOeI1QnU8sNXqwQ7FGnULvKCjt8Tm6eoFEdYeQ0xY',
        'W4': '1lKIAWvs9D8JoyNrZJzB6pU4KVtfWstH3tOEfM9dZMDc',
        'W5': '1nQsQPAoxeRr9QzybjrkpTn5-IuBBCByMNN8txkUVV4w',
        'W6': '1SCV9OBxvHwxQ31t6GFirO6et-raMbMpJdfLjn1_HsgU',
    },
    'sun': {
        'W1': '1-FLTsgge53Wgb3HmIlM-oCOWow4_kLEJ0bey8MaMxVI',
        'W2': '1-WLiRHFXlD5BawHdBkI2kLSlLttBXU4ufnTwLB8pufM',
        'W3': '1EVv6ktg-6ZC5e9UBFvPIVrZI0T2WAbx3OHau_T-OgSE',
        'W4': '1XyXhn_R3VuNcsmHCmCEwojFM1hUPGhQe6FDrfo3ALds',
        'W5': '1YsreB2g0AeDbiFOIu2JOUmbAkwugBRaaf8SMUA2mrg0',
        'W6': '1oU4K52UaKJT3EEvDdOs8o4tD97Wxl4SKSFQK193w2bQ',
    }
}

In [17]:
lang = 'sun'

In [18]:
# Get gold answer
gold_ans = {}
for ref_id in tqdm(workers_ids):
    sh_task1 = client.open_by_key(id_to_key_task1[lang][ref_id])
    wks_task1 = sh_task1.worksheet('title', 'Data')
    gold_col_num  = 6 if lang == 'ind' else 7
    gold_ans[ref_id] = wks_task1.get_col(gold_col_num)[1:]


100%|██████████| 6/6 [00:09<00:00,  1.60s/it]


In [19]:
conflict_data = {}
for pred_id in tqdm(workers_ids):
    all_conflict_counts = []
    sh_pred = client.open_by_key(id_to_key_task2[lang][pred_id])
    for ref_id in workers_ids:
        if pred_id == ref_id:
            all_conflict_counts.append(0)
        else:
            wks_pred = sh_pred.worksheet('title', ref_id)

            # Get answers
            ans_col_num  = 6 if lang == 'ind' else 7
            pred_ans = wks_pred.get_col(ans_col_num)[1:]
            
            # Get conflict status
            stat, stat_buffer = [], []
            for pred, gold in zip(pred_ans, gold_ans[ref_id]):

                stat_buffer.append('OK' if pred == gold else 'CONFLICT')
                if len(stat_buffer) == 5:
                    stat.append('CONFLICT' if 'CONFLICT' in stat_buffer else 'OK')
                    stat_buffer = []

            ok_count, conflict_count = stat.count('OK'), stat.count('CONFLICT')
            if ok_count + conflict_count != 250:
                print(pred_id, ref_id, ok_count, conflict_count)
            assert ok_count + conflict_count == 250
            all_conflict_counts.append(conflict_count)
    conflict_data[pred_id] = all_conflict_counts

100%|██████████| 6/6 [00:23<00:00,  3.84s/it]


In [20]:
conflict_data

{'W1': [0, 8, 1, 4, 7, 9],
 'W2': [1, 0, 4, 4, 4, 10],
 'W3': [1, 4, 0, 2, 3, 4],
 'W4': [0, 6, 5, 0, 6, 7],
 'W5': [6, 12, 12, 10, 0, 22],
 'W6': [6, 12, 15, 2, 17, 0]}

In [21]:
consistency_df = pd.DataFrame.from_dict(conflict_data, orient='index', columns=workers_ids)

In [22]:
consistency_df

,W1,W2,W3,W4,W5,W6
W1,0,8,1,4,7,9
W2,1,0,4,4,4,10
W3,1,4,0,2,3,4
W4,0,6,5,0,6,7
W5,6,12,12,10,0,22
W6,6,12,15,2,17,0
